Nama: Muhammad Rizky Haritama Putra <br>
Nim: 2309106083<br>
Posttest 3

Import Library

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Buat ngebaca file csv dan mengubah nama kolom unnamed menjadi id

In [22]:
df = pd.read_csv("HepatitisCdata.csv")
df = df.rename(columns={'Unnamed: 0': 'ID'})


Ngecek apakah terdapat missing value, duplicate value, dan outlier

In [23]:
print("=== CEK MISSING VALUE ===")
#cek missing value
missing = df.isnull().sum()
print(missing)

=== CEK MISSING VALUE ===
ID           0
Category     0
Age          0
Sex          0
ALB          1
ALP         18
ALT          1
AST          0
BIL          0
CHE          0
CHOL        10
CREA         0
GGT          0
PROT         1
dtype: int64


In [24]:
# Cek Duplicate Value
print("CEK DUPLICATE VALUE")
duplicate_count = df.duplicated().sum()
print(duplicate_count)

CEK DUPLICATE VALUE
0


In [25]:
#Cek Outlier (Metode IQR)
print("CEK OUTLIER (IQR METHOD)")

numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    print(f"{col}: {outliers} outlier")

CEK OUTLIER (IQR METHOD)
ID: 0 outlier
Age: 1 outlier
ALB: 27 outlier
ALP: 10 outlier
ALT: 36 outlier
AST: 64 outlier
BIL: 47 outlier
CHE: 24 outlier
CHOL: 12 outlier
CREA: 12 outlier
GGT: 65 outlier
PROT: 20 outlier


Kode diatas digunakan untuk memeriksa kualitas data sebelum analisis. Pengecekan missing value dilakukan untuk menemukan data kosong yang bisa memengaruhi hasil dan perlu ditangani. Pengecekan duplicate value mendeteksi data ganda yang bisa menimbulkan bias. Pengecekan outlier dengan metode IQR digunakan untuk menemukan nilai ekstrem di luar batas wajar yang dapat mengganggu pola data. Langkah ini penting agar dataset bersih, akurat, dan siap digunakan untuk analisis atau pemodelan.

##1. Data cleaning

In [26]:
#Tangani missing value khusus kolom tertentu
impute_cols = ['ALB', 'ALP', 'ALT', 'CHOL', 'PROT']
print("\n=== IMPUTASI NILAI HILANG ===")

for col in impute_cols:
    if col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            print(f"Kolom {col}: {missing_count} nilai hilang → diganti median = {median_val:.2f}")
        else:
            print(f"Kolom {col}: tidak ada nilai hilang ")
    else:
        print(f"Kolom {col} tidak ditemukan dalam dataset ")


=== IMPUTASI NILAI HILANG ===
Kolom ALB: 1 nilai hilang → diganti median = 41.95
Kolom ALP: 18 nilai hilang → diganti median = 66.20
Kolom ALT: 1 nilai hilang → diganti median = 23.00
Kolom CHOL: 10 nilai hilang → diganti median = 5.30
Kolom PROT: 1 nilai hilang → diganti median = 72.20


In [27]:
# Daftar kolom numerik yang akan diimputasi outliernya
kolom_outlier = ['Age', 'ALB', 'ALP', 'ALT', 'AST',
                 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']

# Fungsi imputasi outlier dengan median
def impute_outlier_iqr(df, kolom):
    Q1 = df[kolom].quantile(0.25)
    Q3 = df[kolom].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    median = df[kolom].median()

    # Ganti nilai di bawah lower dan di atas upper dengan median
    df.loc[df[kolom] < lower, kolom] = median
    df.loc[df[kolom] > upper, kolom] = median

# Terapkan ke semua kolom
for kolom in kolom_outlier:
    before = df[kolom].copy()
    impute_outlier_iqr(df, kolom)
    print(f"{kolom}: imputasi selesai (median={df[kolom].median():.2f})")

print("\nSemua outlier sudah diganti dengan median masing-masing kolom.")

Age: imputasi selesai (median=47.00)
ALB: imputasi selesai (median=41.95)
ALP: imputasi selesai (median=66.20)
ALT: imputasi selesai (median=23.00)
AST: imputasi selesai (median=25.90)
BIL: imputasi selesai (median=7.30)
CHE: imputasi selesai (median=8.26)
CHOL: imputasi selesai (median=5.30)
CREA: imputasi selesai (median=77.00)
GGT: imputasi selesai (median=23.30)
PROT: imputasi selesai (median=72.20)

Semua outlier sudah diganti dengan median masing-masing kolom.


In [28]:
# menyimpan data csv setelah di clean
df.to_csv("data_clean_HepatitisCdata.csv", index=False)

Kode ini menangani missing value dan outlier agar data lebih bersih. Nilai kosong di kolom tertentu diganti dengan median karena lebih stabil terhadap nilai ekstrem. Outlier dideteksi dengan metode IQR lalu diganti median agar distribusi data tetap wajar tanpa menghapus data. Langkah ini membuat dataset lebih konsisten dan siap dianalisis.

##2. Normalisasi / Standarisasi Kolom Numerik

In [29]:
# Baca dataset
df = pd.read_csv("data_clean_HepatitisCdata.csv")

In [30]:
# Normalisasi / standarisasi untuk kolom numerik
scaler = StandardScaler()
num_cols = ['Age','ALB', 'ALP', 'ALT', 'CHOL', 'PROT','AST','BIL','CHE','CREA','GGT']
df[num_cols] = scaler.fit_transform(df[num_cols])

print("\nKolom numerik telah distandarisasi.")
df[num_cols].head()


Kolom numerik telah distandarisasi.


,Age,ALB,ALP,ALT,CHOL,PROT,AST,BIL,CHE,CREA,GGT
0,-1.539675,-0.844293,-0.758827,-1.485073,-2.069345,-0.818096,-0.597660,-0.104753,-0.728265,1.966669,-0.925294
1,-1.539675,-0.844293,0.222981,-0.535580,-0.515308,0.946857,-0.216403,-1.035802,1.634733,-0.286894,-0.690955
2,-1.539675,1.150866,0.465675,1.142164,-0.119375,1.605772,-0.040439,-0.466828,0.336199,0.558192,0.487436
3,-1.539675,0.272046,-0.786406,0.625935,-0.574698,0.758595,-0.524342,2.843567,-0.505341,0.135649,0.527609
4,-1.539675,-0.678029,0.432580,0.810303,-0.990428,-0.888694,-0.201740,0.438358,0.508965,-0.146046,0.266488


In [31]:
#menyimpan data csv setelah di normalisasi
df.to_csv("data_normalisasi_HepatitisCdata.csv", index=False)

Kode ini melakukan standarisasi pada kolom numerik menggunakan StandardScaler. Proses ini mengubah setiap kolom agar memiliki rata-rata 0 dan standar deviasi 1, sehingga skala semua fitur menjadi sebanding. Standarisasi penting karena banyak algoritma machine learning sensitif terhadap perbedaan skala antarfitur. Dengan langkah ini, data menjadi lebih seimbang dan siap digunakan untuk analisis atau pemodelan.

##3. Encoding Kolom Kategorikal

In [32]:
# ngebaca file csv
df = pd.read_csv("data_normalisasi_HepatitisCdata.csv")

In [33]:
# Encode gender (f=0, m=1)
df["Sex"] = df["Sex"].astype(str).str.strip().str.lower().map({"f": 0, "m": 1})
print(df["Sex"].head())



0    1
1    1
2    1
3    1
4    1
Name: Sex, dtype: int64


In [34]:
df["Category"] = (
    df["Category"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({
        "0=blood donor": 0,
        "1=hepatitis": 1,
        "2=fibrosis": 2,
        "3=cirrhosis": 3,
        "0s=suspect blood donor": 4
    })
)
print(df["Category"].unique())

[0 4 1 2 3]


In [35]:
# menyimpan data csv setelah di encod
df.to_csv("data_encod_HepatitisCdata.csv", index=False)

Kode ini melakukan encoding pada kolom kategorik agar dapat diproses sebagai data numerik. Pada kolom Sex, nilai 'f' diubah menjadi 0 dan 'm' menjadi 1 agar bisa digunakan dalam analisis atau model. Sedangkan pada kolom Category, setiap kategori teks (seperti blood donor, hepatitis, fibrosis, dll.) dipetakan ke angka tertentu. Proses ini penting karena algoritma machine learning tidak dapat memproses data berupa teks, sehingga perlu diubah menjadi format numerik yang konsisten.

##4. Feature Engineering

In [36]:
#ngebaca file csv
df = pd.read_csv("data_encod_HepatitisCdata.csv")

In [37]:
# Rasio AST/ALT versi aman
df["AST_ALT_ratio"] = df["AST"] / (df["ALT"] + 1e-6)

# Gunakan nilai absolut agar tidak ada log negatif
df["log_AST_ALT_ratio"] = np.log(np.abs(df["AST_ALT_ratio"]) + 1e-6)

# Cek hasil kolom baru
print(df[["AST", "ALT", "AST_ALT_ratio", "log_AST_ALT_ratio"]].head())

# Cek korelasi dengan target Category
if "Category" in df.columns:
    corr = df[["AST_ALT_ratio", "log_AST_ALT_ratio", "Category"]].corr()
    print("\nKorelasi terhadap target Category:")
    print(corr["Category"].sort_values(ascending=False))

# Simpan dataset baru ke file CSV
df.to_csv("data_feature_HepatitisCdata.csv", index=False)

        AST       ALT  AST_ALT_ratio  log_AST_ALT_ratio
0 -0.597660 -1.485073       0.402445          -0.910193
1 -0.216403 -0.535580       0.404055          -0.906202
2 -0.040439  1.142164      -0.035405          -3.340870
3 -0.524342  0.625935      -0.837692          -0.177104
4 -0.201740  0.810303      -0.248968          -1.390427

Korelasi terhadap target Category:
Category             1.000000
AST_ALT_ratio       -0.024096
log_AST_ALT_ratio   -0.298636
Name: Category, dtype: float64


Kode ini membuat fitur baru untuk analisis. Kolom AST_ALT_ratio dihitung sebagai rasio AST / ALT untuk melihat keseimbangan kedua enzim hati. Ditambahkan nilai kecil 1e-6 agar tidak terjadi pembagian nol. Lalu dibuat kolom log_AST_ALT_ratio dengan logaritma dari nilai rasio untuk menstabilkan sebaran data dan menghindari log negatif.
Selanjutnya, dilakukan korelasi antara fitur baru dengan kolom target Category untuk mengecek seberapa besar hubungan rasio ini dengan label penyakit. Terakhir, dataset yang sudah diperbarui disimpan ke file CSV agar bisa digunakan untuk analisis atau pemodelan berikutnya.

##5. Splitting Data ke Training & Testing

In [38]:
#ngebaca file csv
df = pd.read_csv("data_feature_HepatitisCdata.csv")

In [39]:
# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=["Category"])   # semua kolom kecuali target
y = df["Category"]                  # target (label kelas)

# Split data: 80% training, 20% testing (bisa disesuaikan)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Tampilkan hasil
print("Ukuran dataset keseluruhan:", df.shape)
print("Training set:", X_train.shape)
print("Testing set:", X_test.shape)

Ukuran dataset keseluruhan: (615, 16)
Training set: (492, 15)
Testing set: (123, 15)


Kode ini digunakan untuk memisahkan data menjadi fitur dan target, lalu membagi dataset menjadi data latih dan data uji.
Pertama, variabel X berisi semua kolom kecuali Category sebagai fitur, sedangkan y hanya berisi kolom Category sebagai target (label). Kemudian, train_test_split digunakan untuk membagi data menjadi 80% untuk training dan 20% untuk testing dengan parameter stratify=y agar distribusi kelas target tetap seimbang di kedua subset.
Langkah ini penting agar model dapat belajar dari data latih dan diuji performanya pada data uji yang belum pernah dilihat, sehingga hasil evaluasi menjadi lebih objektif.